In [1]:
import sys

!{sys.executable} -m pip install --upgrade stepfunctions

In [2]:
import uuid
import logging
import stepfunctions
import boto3
import sagemaker

from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from stepfunctions import steps
from stepfunctions.steps import TrainingStep, ModelStep
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow

# Set up sagemaker connection session
session = sagemaker.Session()
# Set up logging level
stepfunctions.set_stream_logger(level=logging.INFO)

# Set up AWS python SDK session
region = boto3.Session().region_name
# Set up S3 bucket location 
bucket = session.default_bucket()
# Generate random UUID for this run
id = uuid.uuid4().hex

# Create a unique name for the AWS Glue job to be created. If you change the
# default name, you may need to change the Step Functions execution role.
job_name = "glue-customer-churn-etl-{}".format(id)

# Create a unique name for the AWS Lambda function to be created. If you change
# the default name, you may need to change the Step Functions execution role.
function_name = "query-training-status-{}".format(id)

/Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/14/24 18:23:08] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=752484;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=211606;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/pengzhao/Library/Application Support/sagemaker/config.yaml


[12/14/24 18:23:09] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=769806;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=442805;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

In [3]:
# paste the role'ARN we created for this course 
workflow_execution_role = "arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1"

# SageMaker Execution Role
# You can use sagemaker.get_execution_role() if running inside sagemaker's notebook instance
#sagemaker_execution_role = (sagemaker.get_execution_role())

sagemaker_execution_role = "arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1"

In [4]:
session = sagemaker.Session()
# In the case you cannot get the right default bucket, you can specify a S3 location you created manually
bucket = session.default_bucket()
print(bucket)

[12/14/24 18:42:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=60274;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=586291;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker-us-west-1-798223350085


In [5]:
# Same role we created in homework 1
glue_role = "arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1"

In [6]:
# Same role we created in homework 1
lambda_role = "arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1"

In [7]:
# Name anything you want
project_name = "ml_deploy"

# Copy customer churn csv data into this notebook instance
# Then use the following code to copy your local CSV to S3 location for model training
data_source = S3Uploader.upload(
    local_path="./data/customer-churn.csv",
    desired_s3_uri="s3://{}/{}".format(bucket, project_name),
    sagemaker_session=session,
)

train_prefix = "train"
val_prefix = "validation"

# Train and validation dataset location in S3
train_data = "s3://{}/{}/{}/".format(bucket, project_name, train_prefix)
validation_data = "s3://{}/{}/{}/".format(bucket, project_name, val_prefix)

# ETL: split data into training and validation

In [8]:
# Copy glue_etl.py to notebook instance
# Upload glue script to S3 bucket
glue_script_location = S3Uploader.upload(
    local_path="./code/glue_etl.py",
    desired_s3_uri="s3://{}/{}".format(bucket, project_name),
    sagemaker_session=session,
)
glue_client = boto3.client("glue")

# create a ETL job in Glue to split training and validation dataset
response = glue_client.create_job(
    Name=job_name,
    Description="PySpark job to extract the data and split in to training and validation data sets",
    Role=glue_role,  # you can pass your existing AWS Glue role here if you have used Glue before
    ExecutionProperty={"MaxConcurrentRuns": 2},
    Command={"Name": "glueetl", "ScriptLocation": glue_script_location, "PythonVersion": "3"},
    DefaultArguments={"--job-language": "python"},
    GlueVersion="3.0",
    WorkerType="Standard",
    NumberOfWorkers=2,
    Timeout=60,
)

[12/14/24 18:42:21] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=3120;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=692957;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

In [9]:
import os
os.listdir('./code')

['query_training_status.py', 'glue_etl.py']

In [14]:
import zipfile

# Model validation pipeline
# Copy query_training_status.py to lcoal instance notebook
zip_name = "query_training_status.zip"
lambda_source_code = "./code/query_training_status.py"

# Zip the script
zf = zipfile.ZipFile(zip_name, mode="w")
zf.write(lambda_source_code, arcname=lambda_source_code.split("/")[-1])
zf.close()

# Copy zipped script to S3 for lambda use
S3Uploader.upload(
    local_path=zip_name,
    desired_s3_uri="s3://{}/{}".format(bucket, project_name),
    sagemaker_session=session,
)

's3://sagemaker-us-west-1-798223350085/ml_deploy/query_training_status.zip'

In [15]:
# Create lambda client
lambda_client = boto3.client("lambda")

# Create a lambda function for model result validation
response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime="python3.9",
    Role=lambda_role,
    Handler="query_training_status.lambda_handler",
    Code={"S3Bucket": bucket, "S3Key": "{}/{}".format(project_name, zip_name)},
    Description="Queries a SageMaker training job and return the results.",
    Timeout=15,
    MemorySize=128,
)

In [16]:
# Retrive XGBoost algorithm container for training purpose
container = sagemaker.image_uris.retrieve("xgboost", region, "latest")

# Create XGBoost estimator (model) with m4.xlarge instance type
xgb = sagemaker.estimator.Estimator(
    container,
    sagemaker_execution_role,
    train_instance_count=1,
    train_instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, project_name),
)

# Set initial hyperparameter configurations
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective="binary:logistic",
    eval_metric="error",
    num_round=100,
)

In [17]:
# Specify model training execution configurations
# SageMaker expects unique names for each job, model and endpoint.
# If these names are not unique the execution will fail.
execution_input = ExecutionInput(
    schema={
        "TrainingJobName": str,
        "GlueJobName": str,
        "ModelName": str,
        "EndpointName": str,
        "LambdaFunctionName": str,
    }
)

In [18]:
# Start glue data ETL job run
etl_step = steps.GlueStartJobRunStep(
    "Extract, Transform, Load",
    parameters={
        "JobName": execution_input["GlueJobName"],
        "Arguments": {
            "--S3_SOURCE": data_source,
            "--S3_DEST": "s3a://{}/{}/".format(bucket, project_name),
            "--TRAIN_KEY": train_prefix + "/",
            "--VAL_KEY": val_prefix + "/",
        },
    },
)

In [19]:
# Define model training setp
training_step = steps.TrainingStep(
    "Model Training",
    estimator=xgb,
    data={
        "train": TrainingInput(train_data, content_type="text/csv"),
        "validation": TrainingInput(validation_data, content_type="text/csv"),
    },
    job_name=execution_input["TrainingJobName"],
    wait_for_completion=True,
)

In [20]:
# Define model store / register step
model_step = steps.ModelStep(
    "Save Model",
    model=training_step.get_expected_model(),
    model_name=execution_input["ModelName"],
    result_path="$.ModelStepResults",
)

In [21]:
# Define lambda for model validation step
lambda_step = steps.compute.LambdaStep(
    "Query Training Results",
    parameters={
        "FunctionName": execution_input["LambdaFunctionName"],
        "Payload": {"TrainingJobName.$": "$.TrainingJobName"},
    },
)

In [22]:
# Name accuracy check in AWS StepFunction
check_accuracy_step = steps.states.Choice("Accuracy > 90%")

In [23]:
# Configure model deployment endpoint
endpoint_config_step = steps.EndpointConfigStep(
    "Create Model Endpoint Config",
    endpoint_config_name=execution_input["ModelName"],
    model_name=execution_input["ModelName"],
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
)

In [24]:
# Update Model Endpoint
endpoint_step = steps.EndpointStep(
    "Update Model Endpoint",
    endpoint_name=execution_input["EndpointName"],
    endpoint_config_name=execution_input["ModelName"],
    # If you want continuous training in existing pipeline, need to modify this to true
    update=False,
)

In [25]:
# Name fail critera in AWS StepFunction
fail_step = steps.states.Fail(
    "Model Accuracy Too Low", comment="Validation accuracy lower than threshold"
)

In [26]:
# Define logic for model validation
threshold_rule = steps.choice_rule.ChoiceRule.NumericLessThan(
    variable=lambda_step.output()["Payload"]["trainingMetrics"][0]["Value"], value=0.1
)

check_accuracy_step.add_choice(rule=threshold_rule, next_step=endpoint_config_step)
check_accuracy_step.default_choice(next_step=fail_step)

In [27]:
# Define step function end
endpoint_config_step.next(endpoint_step)

Update Model Endpoint EndpointStep(resource='arn:aws:states:::sagemaker:createEndpoint', parameters={'EndpointConfigName': <stepfunctions.inputs.placeholders.ExecutionInput object at 0x7f90e16364c0>, 'EndpointName': <stepfunctions.inputs.placeholders.ExecutionInput object at 0x7f90e1636eb0>}, type='Task')

In [28]:
# Chain model training automation as a pipeline
workflow_definition = steps.Chain(
    [etl_step, training_step, model_step, lambda_step, check_accuracy_step]
)

In [29]:
# Define workflow in AWS StepFunction
workflow = Workflow(
    name="MyInferenceRoutine_{}".format(id),
    definition=workflow_definition,
    role=workflow_execution_role,
    execution_input=execution_input,
)

In [30]:
# Genearate DAG in graph in AWS StepFunction
workflow.render_graph()

In [31]:
# Create workflow in AWS StepFunction
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:us-west-1:798223350085:stateMachine:MyInferenceRoutine_ba65a86c01c8407ba75b475abdc045ec'

In [32]:
# Execute training automation workflow and pass parameters
execution = workflow.execute(
    inputs={
        "TrainingJobName": "regression-{}".format(id),  # Each Sagemaker Job requires a unique name,
        "GlueJobName": job_name,
        "ModelName": "CustomerChurn-{}".format(id),  # Each Model requires a unique name,
        "EndpointName": "CustomerChurn",  # Each Endpoint requires a unique name
        "LambdaFunctionName": function_name,
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.


In [33]:
execution.render_progress()

In [34]:
execution.list_events()

[{'timestamp': datetime.datetime(2024, 9, 3, 13, 55, 1, 964000, tzinfo=tzlocal()),
  'type': 'ExecutionStarted',
  'id': 1,
  'previousEventId': 0,
  'executionStartedEventDetails': {'input': '{\n    "TrainingJobName": "regression-ba65a86c01c8407ba75b475abdc045ec",\n    "GlueJobName": "glue-customer-churn-etl-ba65a86c01c8407ba75b475abdc045ec",\n    "ModelName": "CustomerChurn-ba65a86c01c8407ba75b475abdc045ec",\n    "EndpointName": "CustomerChurn",\n    "LambdaFunctionName": "query-training-status-ba65a86c01c8407ba75b475abdc045ec"\n}',
   'inputDetails': {'truncated': False},
   'roleArn': 'arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1'}},
 {'timestamp': datetime.datetime(2024, 9, 3, 13, 55, 2, 17000, tzinfo=tzlocal()),
  'type': 'TaskStateEntered',
  'id': 2,
  'previousEventId': 0,
  'stateEnteredEventDetails': {'name': 'Extract, Transform, Load',
   'input': '{\n    "TrainingJobName": "regression-ba65a86c01c8407ba75b475abdc045ec",\n    "GlueJobName": "glue-customer-churn-etl-ba65a

In [35]:
workflow.list_executions(html=True)

Name,Status,Started,End Time
936b1199-e076-49a4-ac90-371b554e5721,RUNNING,"Sep 03, 2024 01:55:01.964 PM",-


In [43]:
# Validate endpoint
endpoint_name="CustomerChurn"
sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name='us-west-2')

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Body = "2.0,400.0,0.38571846040122537,2.0,4.177940384158745,0.0,3.745462710628048,250.0,3.699591756294294,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0"
)
print(response['Body'].read().decode('utf-8'))

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint CustomerChurn of account 798223350085 not found.

In [44]:
# Clean up resource, note does not include delete SageMaker endpoint
lambda_client.delete_function(FunctionName=function_name)
glue_client.delete_job(JobName=job_name)
workflow.delete()

[INFO] Workflow has been marked for deletion. If the workflow has running executions, it will be deleted when all executions are stopped.


In [45]:
!pwd

/Users/pengzhao/Documents/DataScience/mlops/MLOpsCamp/week3/ETL
